# NEXRAD HCA Tutorial

---

This interactive tutorial takes you through the steps of how to run RadxHca. RadxHca runs the hydrometeor classification algorithm described in [Park et al. (2009)](https://doi.org/10.1175/2008WAF2222205.1). 

From Section 2a of the manuscript: 
> The HCA discriminates between 10 classes of radar echo: 1) ground clutter, including that due to anomalous propagation (GC/AP); 2) biological scatterers (BS); 3) dry aggregated snow (DS); 4) wet snow (WS); 5) crystals of various orientations (CR); 6) graupel (GR); 7) “big drops” (BD); 8) light and moderate rain (RA); 9) heavy rain (HR); and 10) a mixture of rain and hail (RH). The big-drops category designates rain with a drop size distribution skewed toward large raindrops. This usually implies the presence of drops bigger than 3 mm and the deficit of smaller drops.

---

*Note: this tutorial is just one type of workflow to use RadxHca.*

---

## Tutorial Overview
### 1. Setup

#### Directory organization

The structure of the echo tutorial on JupyterHub is shown in the diagram below. The parent or base directory is "lrose-hub" and contains all of the notebooks, parameter files, and data for the workshop.

<div>
<img src="../images/nexrad_hca_structure.png" width="500"/>
</div>

#### Download raw data and prepare parameter files

Raw data files:
* gfsanl_4_2025061706.g2.tar
* gfsanl_4_2025061712.g2.tar
* KCYS20250617_082327_V06

Parameter files:

* Grib2toMdv params
* Mdv2SoundingSpdb params
* Pid Thresholds params (S-band, simultaneous transmit)
* RadxHca params


### 2. Prepare data for analysis

* Convert Nexrad level 2 to CfRadial
    * RadxConvert
* Retrieve sounding from GFS analysis, which is needed to estimate the melting level in the HCA/PID analysis
    * Grib2toMdv, Mdv2SoundingSpdb

### 3. Run RadxHca

* Run NEXRAD HCA and NCAR PID algorithms
    * RadxHca

### 4. Plot HCA

* Visualize results of RadxHca analysis using [Py-ART](https://arm-doe.github.io/pyart/)

### 5. Note on task cells

This notebook uses two colored cells to indicate tasks.

<div class="alert alert-block alert-info"> <b>File Task: modify parameters in text files.</b> 

These text blocks help the user modify the parameter files or other functions in *external* text files.

</div>

<div class="alert alert-block alert-warning"> <b>Cell Task: run a command in Jupyter notebook cell.</b> 

These text blocks instruct the users to run a command *in* a cell within the Jupyter notebook. If you prefer, you are welcome to copy the commands (minus the ! symbol) into a terminal window.

</div>

---

# 1. Setup
## Environment and packages

First, we import the required python packages to run this notebook. Most of the LROSE processing can be done with the os package and shell commands. At the end we will plot the output using Py-ART.

In [ ]:
import os
import pyart
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np

Next, we set up the directory structure to simplify our commands. If you are running this notebook on the LROSE Gateway JupyterHub, these paths go to the parent directory containing all the workshop resources and the LROSE binaries. 

**If you have downloaded this notebook, please modify BASE_DIR and RAW_DIR to work on your personal machine. Note that raw data is currently provided through the Gateway. We're working to set up more permanent storage for offline users - in the meantime, contact the Gateway team to request the data.**

* BASE_DIR: the base directory containing the directories for the notebooks, data, parameter files
* RAW_DIR: the directory containing the raw data used in tutorials
* RADAR_NAME: the name of the radar used in this tutorial

In [ ]:
os.environ['BASE_DIR'] = '/home/jovyan/lrose-hub'
os.environ['RAW_DIR'] = '/home/jovyan/share/raw'
os.environ['RADAR_NAME'] = 'KCYS'
base_dir = os.environ['BASE_DIR']
radar_name = os.environ['RADAR_NAME']
!echo "Base directory: "$BASE_DIR
!echo "Radar name: "$RADAR_NAME

## Set up directories

We need to set up the required data directories. The raw radar data and GFS analysis is located within the share directory on the JupyterHub. We delete any existing files and directories specific to this tutorial to ensure we're starting with clean directories and files.

In [ ]:
## make a directory for all the data files in the HCA tutorial (raw and analysis)
!rm -rf ${BASE_DIR}/data/nexrad_hca
!mkdir -p ${BASE_DIR}/data/nexrad_hca

## make subdirectory within data for the converted data
!rm -rf ${BASE_DIR}/data/nexrad_hca/cfradial
!mkdir ${BASE_DIR}/data/nexrad_hca/cfradial

## make subdirectory within data for sounding data
!rm -rf ${BASE_DIR}/data/nexrad_hca/sounding
!mkdir ${BASE_DIR}/data/nexrad_hca/sounding

## make a directory for the parameter files
!rm -rf ${BASE_DIR}/params/nexrad_hca
!mkdir -p ${BASE_DIR}/params/nexrad_hca


We can examine the raw radar file using RadxPrint. RadxPrint reads CfRadial and other raw radar formats and prints out radar metadata, variable names, and sweep information. For example, we can look at the variables using the following command, piping the output into the bash head command.

In [ ]:
# print out the first 50 lines of RadxPrint output
!RadxPrint -f ${RAW_DIR}/nexrad_hca/KCYS20250617_082327_V06 | head -50


# 2. Prepare data for analysis
## 2.1 Convert Nexrad Level II to CfRadial files

We convert the raw to CfRadial using RadxConvert so the other applications can read the files. Note that in the vast majority of cases, RadxConvert can be run without a parameter file, including in this case. However, there are instances where a parameter file is useful.

Here, the *-f* flag provides the path to the raw radar file and the *-outdir* flag tells RadxConvert where the CfRadial file will be written. 

In [ ]:
!RadxConvert -f ${RAW_DIR}/nexrad_hca/KCYS20250617_082327_V06 -outdir ${BASE_DIR}/data/nexrad_hca/cfradial


## 2.2 Visually Inspect Data

First, let's look at a basic plot of the raw data using Py-ART.

In [ ]:
# Read CfRadial file into radar object
inDir_raw = base_dir+"/data/nexrad_hca/cfradial/20250617/"
file_raw = "cfrad.20250617_082327.974_to_20250617_083013.086_KCYS_SUR.nc"
radar_raw = pyart.io.read_cfradial(inDir_raw+file_raw)

# Plot raw data

displayRaw = pyart.graph.RadarDisplay(radar_raw)
figRaw = plt.figure(1, (12, 5))

# DBZ (input)

axDbz = figRaw.add_subplot(121)
displayRaw.plot_ppi('REF', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
displayRaw.plot_range_rings([50, 100, 150, 200, 250])
displayRaw.plot_cross_hair(250.)
displayRaw.set_limits(xlim=(-300,300),ylim=(-300,300))

# ZDR

axZdr = figRaw.add_subplot(122)
displayRaw.plot_ppi('ZDR', 0, vmin=-1., vmax=5.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="ZDR",
    cmap="nipy_spectral")
displayRaw.plot_range_rings([50, 100, 150, 200, 250])
displayRaw.plot_cross_hair(250.)
displayRaw.set_limits(xlim=(-300,300),ylim=(-300,300))

## 2.3 Untar GFS analysis grib2 files for sounding
We use the GFS analysis to obtain a sounding for the PID algorithm. We need to untar the files, convert to a gridded file format (Grib2toMdv), then extract the sounding and store in in the Spdb format (Mdv2SoundingSpdb).

To download GFS analyses for other cases or radars, please visit [this link](https://www.ncei.noaa.gov/products/weather-climate-models/global-forecast) and download it however you prefer. Using the NCEI servers and the HAS option usually works well.

In [ ]:
# Grib2Mdv --> Mdv2SoundingSpdb

# make directory for grib files in data/sounding
!rm -rf ${BASE_DIR}/data/nexrad_hca/sounding/grib
!mkdir -p ${BASE_DIR}/data/nexrad_hca/sounding/grib

# extract file from tape archive (tar) and place in the grib directory
!tar -xvf ${RAW_DIR}/nexrad_hca/gfsanl_4_2025061706.g2.tar -C ${BASE_DIR}/data/nexrad_hca/sounding/grib/
!tar -xvf ${RAW_DIR}/nexrad_hca/gfsanl_4_2025061712.g2.tar -C ${BASE_DIR}/data/nexrad_hca/sounding/grib/


## 2.4 Convert from Grib2 (model analysis) to MDV

The program to retrieve the model sounding requires a gridded file format called MDV. Grib2toMdv converts the GFS analysis from GRIB to MDV. Grib2toMdv needs to know where to write the gridded data files, the name of the fields, how the grid should be remapped (e.g., map projection), the size of the new grid, and the vertical levels.

While users can generate their own parameter file, frequent users of LROSE will often use existing parameter files from previous projects, particularly if the same model is used for sounding extraction. We have provided a generic parameter file used with GFS analyses, where the model field names have already been filled out and specifies that only the non-forecast times should be written.

| Parameter | Description |
| --- | --- |
|write_forecast|Specify if output should be written in a forecast directory structure.|
|write_non_forecast|Specify if output should be written in a non-forecast directory structure.|
|non_forecast_mdv_url|Absolute path where MDV files are written.|

<div class="alert alert-block alert-warning"> <b>Cell Task: copy over the Grib2toMdv parameter file.</b> 

Copy the parameter file: <code lang="bash">!rsync -av /home/jovyan/share/raw/shared_params/Grib2toMdv_params_gfs ${BASE_DIR}/params/nexrad_hca/Grib2toMdv_params</code>

</div>


<div class="alert alert-block alert-info"> <b>File Task: verify and set output parameters</b> 
    <br>

1. In the Grib2toMdv parameter file (Grib2toMdv_params), verify that <code lang="bash">write_forecast = FALSE;</code>

2. In the Grib2toMdv parameter file (Grib2toMdv_params), verify that <code lang="bash">write_non_forecast = TRUE;</code>

3. In the Grib2toMdv parameter file (Grib2toMdv_params), change <code lang="bash">non_forecast_mdv_url = "/home/jovyan/lrose-hub/data/nexrad_hca/sounding/mdv";</code>

</div>

---

Next, we specify the output grid.

| Parameter | Description |
| --- | --- |
|remap_output|Specify whether to remap the output (*TRUE* or *FALSE*, usually *TRUE*).|
|out_projection_info|Set map projection type and radar location if necessary.|
|out_grid_info|User-specified domain surrounding the region of interest and radar(s). Parameters: nx/ny - number of points, minx/miny - bottom left lat/lon (deg), dx/dy - delta lon/lat (deg).|
|height_levels|User-specified height levels, optional.|

<div class="alert alert-block alert-info"> <b>File Task: specify the output grid.</b> 

1. In the Grib2toMdv parameter file (Grib2toMdv_params), change <code lang="bash">remap_output = TRUE;</code>
    
2. In the Grib2toMdv parameter file (Grib2toMdv_params), make sure that <code lang="bash">type = PROJ_LATLON,</code> in <code lang="bash">out_projection_info</code>.

3. In the Grib2toMdv parameter file (Grib2toMdv_params), change the grid dimensions. These values specify a 60x60 grid, with a grid spacing of 0.1º in each direction, where the lower left hand corner of the box is 38ºN, 107ºW.


<code lang="bash">out_grid_info = {
    nx = 60,
    ny = 60,
    minx = -107,
    miny = 38,
    dx = 0.1,
    dy = 0.1
};</code>


</div>

---
**Now we can run Grib2toMdv to convert the files from Grib to NCAR's MDV format.**

Here, the *-params* flag directs Grib2toMdv to the parameter file and *-f* points to the raw GRIB file path. This can also be specified in the parameter *input_dir*.

In [ ]:
# Grib2 -> MDV
!Grib2toMdv -params ${BASE_DIR}/params/nexrad_hca/Grib2toMdv_params -f ${BASE_DIR}/data/nexrad_hca/sounding/grib/*000.grb2


## 2.5 Convert from MDV (gridded data) to SoundingSpdb

Now that we have model analysis in a small grid around the radar(s), we can extract the sounding from the radar location and store it in a database format (Spdb). Mdv2SoundingSpdb needs the variable names in the MDV files, the geographic location of the radars, and the path where the Spdb files will be writen.

While users can generate their own parameter file, frequent users of LROSE will often use existing parameter files from previous projects, particularly if the same model is used for sounding extraction. We have provided a generic parameter file used with GFS analyses, where the model field names have already been filled out and specifies that only the non-forecast times should be written.

| Parameter | Line # | Description |
| --- | --- | --- |
|station_locations| 247 |Geographic location of the radar(s): radar lat, lon, altitude.|
|ouput_url| 268 |Absolute path where Spdb files are written.|

<div class="alert alert-block alert-warning"> <b>Cell Task: copy over the Grib2toMdv parameter file.</b> 

Copy the parameter file: <code lang="bash">!rsync -av /home/jovyan/share/raw/shared_params/Mdv2SoundingSpdb_params_gfs ${BASE_DIR}/params/nexrad_hca/Mdv2SoundingSpdb_params</code>

</div>


<div class="alert alert-block alert-info"> <b>File Task: Fill in the station location info and set the Spdb output path.</b> 
    <br>
   
1. In the Mdv2SoundingSpdb parameter file (Mdv2SoundingSpdb_params), fill in the <code lang="bash">station_locations</code> parameter for KCYS

2. In the Mdv2SoundingSpdb parameter file (Mdv2SoundingSpdb_params), change <code lang="bash">ouput_url = "/home/jovyan/lrose-hub/data/nexrad_hca/sounding/spdb";</code>


Hint: we can use RadxPrint to get the metadata and obtain the location information for the radar.

<code lang="bash">!RadxPrint -f ${BASE_DIR}/data/nexrad_hca/cfradial/20250617/*.nc | head -30</code>

</div>

---

**Now we can run Mdv2SoundingSpdb to extract the soundings from the MDV files and place them in the Spdb database files.**

Here, the <code lang="bash">-params</code> flag directs Mdv2SoundingSpdb to the parameter file and <code lang="bash">-f</code> points to the raw GRIB file.

In [ ]:
# convert Mdv to Spdb
!Mdv2SoundingSpdb -params ${BASE_DIR}/params/nexrad_hca/Mdv2SoundingSpdb_params -f ${BASE_DIR}/data/nexrad_hca/sounding/mdv/20250617/*.mdv*


We can check the sounding quality using SpdbQuery, which prints out a summary of the sounding data. Here, <code lang="bash">-url</code> indicates the directory where the Spdb files are located and <code lang="bash">-start</code> and <code lang="bash">-end</code> provide the date range over which we want to inspect the sounding information. 

In [ ]:
!SpdbQuery -url ${BASE_DIR}/data/nexrad_hca/sounding/spdb -start "2025 06 17 06 00 00" -end " 2025 06 17 12 00 00"


# 3. Setup and run RadxHca

## RadxHca Overview

With the radar data and sounding, we can now set up and run RadxHca. First, KDP is calculated. Then polarimetric radar data, calculated KDP, and sounding data are used to run the NEXRAD HCA, which determines the dominant hydrometeor at each range gate. 

Running RadxHca requires one main parameter file on the command line. We will guide you through the setup.

RadxHca requires the field names in the input CfRadial file, whether SNR and LDR are available, the method for KDP calculation, how to calculate the NEXRAD HCA and NCAR PID, whether to use attenuation-correction fields, and the variables to be written. 

<div class="alert alert-block alert-warning"> <b>Cell Task: Check command line options of RadxHca</b> 
    <br>
    LROSE applications often let you link to the input files and set the output directory on the command line. Let's check that those options exist (e.g., -f and -outdir). Type the following command into the open cell below and run it!
    <br>
    <code lang="bash">!RadxHca -h</code>
</div>

## 3.1 Generate the parameter files
First, we need to generate the parameter file! LROSE applications use the <code lang="bash">-print_params</code> flag to generate parameter files with the defaults. We just need to specify the location and name of the parameter file. 

RadxHca requires 1 parameter file that sets variable names and file paths.

<div class="alert alert-block alert-warning"> <b>Cell Task: Create the RadxRate parameter files.</b> 
<br>
Type the following commands into the open cell below and run it!
<br>
<code lang="bash">!RadxHca -print_params > ${BASE_DIR}/params/nexrad_hca/RadxHca_params</code>
</div>

## 3.2 Fill out the RadxHca parameter file

Now we will step through and edit the parameter file.

First, we'll start with the parameters associated with the necessary file paths. Since we can link to the input files and output directory on the command line, we don't need to modify them here, but we've listed them for reference.

| Parameter | Description |
| --- | --- |
|input_dir|Path to input data, if not already specified on the command line.|
|output_dir|Path where output files are written, if not already specified on command line with -outdir.|


---

Next, we'll look at the raw file to find which variables are available and what the variable names are.

| Parameter | Description |
| --- | --- |
|SNR_available|Specify if SNR is available.|
|SNR_field_name|SNR variable name.|
|DBZ_field_name|DBZ variable name.|
|ZDR_field_name|ZDR variable name.|
|PHIDP_field_name|PHIDP variable name.|
|RHOHV_field_name|RHOHV variable name.|
|LDR_available|Specify if LDR is available.|
|LDR_field_name|LDR variable name.|

<div class="alert alert-block alert-warning"> <b>Cell Task: grab the variable names for SNR, DBZ, ZDR, PHIDP, RHOHV, and LDR (if present).</b> 

Examine the field names for SNR, DBZ, ZDR, PHIDP, RHOHV, and LDR, if the variables are present in the file.
    
We can use ncdump or RadxPrint to examine the variable names in our radar files. To reduce the application output, you can also pipe information to commands like <code lang="bash">grep</code> or <code lang="bash">head</code>. 
 
<i>Use **one** of the ncdump or RadxPrint commands below in the next Jupyter cell. If you prefer to run them in a terminal window, remove the "!" symbol, which is just for Jupyter notebooks.</i>

<code lang="bash">!ncdump -h ${RAW_DIR}/data/nexrad_hca/cfradial/cfrad*.nc </code>

<code lang="bash">!RadxPrint -f ${RAW_DIR}/data/nexrad_hca/cfradial/cfrad*.nc</code>

Hint: NEXRAD often uses different field names than other radars.

</div>

<div class="alert alert-block alert-info"> <b>File Task: modify the parameter file to specify the correct field names.</b> 

Use the output from the previous cell to fill out the following parameters. 

1. Specify whether spectrum width, normalized coherent power, signal to noise ratio, and linear depolarization ratio are available by setting <code lang="bash">WIDTH_available</code>, <code lang="bash">NCP_available</code>, <code lang="bash">SNR_available</code>, <code lang="bash">LDR_available</code> to <code lang="bash">TRUE</code> or <code lang="bash">FALSE</code>, depending on which is accurate.
2. Add the variable names to <code lang="bash">DBZ_field_name</code>, <code lang="bash">VEL_field_name</code>, <code lang="bash">WIDTH_field_name</code>, <code lang="bash">ZDR_field_name</code>, <code lang="bash">PHIDP_field_name</code>, and <code lang="bash">RHOHV_field_name</code>.

</div>

    
**KDP Parameters**

Usually, the default settings for KDP calculation are fine, but two commonly modified parameters are the filter length and the method for handling phase shift on backscatter. For more information on the KDP calculation in LROSE, please refer to the [LROSE Wiki](http://wiki.lrose.net/index.php/KDP_estimation).

| Parameter | Description |
| --- | --- |
|KDP_fir_filter_len|Filter length used for KDP calculation.|
|KDP_psob_method|Specify method to remove phase shift on backscatter.|
          
We will use the defaults in this example, so no need to modify these in the parameter file.

---

RadxHca allows for use of attenuation-corrected fields in the HCA calculations. This is less of an issue with KCYS, which is an S-band radar, but it's good practice to account for attenutation in the HCA.

| Parameter | Description |
| --- | --- |
|apply_precip_attenuation_correction|Specify whether to use attenuation-corrected DBZ and ZDR in HCA.|
|specify_coefficients_for_attenuation_correction|Specify whether to use user-provided coefficients.|

<div class="alert alert-block alert-info"> <b>File Task: use attenuation-corrected fields.</b> 

1. In the RadxHca main parameter file (RadxHca_params), change <code lang="bash">apply_precip_attenuation_correction = TRUE;</code>

2. In the RadxHca main parameter file (RadxHca_params), change <code lang="bash">specify_coefficients_for_attenuation_correction = FALSE;</code> to use the default coefficients for S-band.

</div>

**HCA and PID Parameters**

The HCA fuzzy logic parameters are all listed within the main RadxHca_params file and the defaults are taken directly from Park et al. (2009), so they do not need to be modified. Since RadxHca can also run the NCAR PID algorithm, we will run that as well for comparison. For additional details on how to run the NCAR PID, please refer to our other tutorials: RadxRate_tutorial, SEAPOL_RadxRate_tutorial, and Guided_Echo_tutorial. 

The most important components of the PID algorithm are the PID thresholds file that sets the fuzzy logic relationships and a relevant sounding for estimating the melting level (i.e., distinguishing between liquid and frozen hydrometeors). For more information on the fuzzy logic method used in the NCAR PID algorithm, please refer to the [LROSE Wiki](http://wiki.lrose.net/index.php/RadxPid_fuzzylogic).

The PID thresholds files, particularly for S-band radars, have been tested in many environments, so those are provided for users on the LROSE wiki.

Soundings for the NCAR PID are either provided through 1) database files for each radar location and at a user-specified temporal frequency, or 2) a single sounding that is added to the PID thresholds file. Here, we will use the same soundings taken from the GFS analysis as in the NEXRAD HCA.

| Parameter | Description |
| --- | --- |
|pid_thresholds_file_path|Path to fuzzy logic PID thresholds file.|
|pid_override_temp_profile|Specify whether soundings are in Spdb format, otherwise sounding found in fuzzy logic file.|
|sounding_spdb_url|Path to Spdb soundings.|
|sounding_location_name|Name of sounding location.|

We will use the Spdb sounding files that we created earlier in the tutorial.

<div class="alert alert-block alert-warning"> <b>Task: download and move the PID thresholds file.</b> 

1. Download the S-band PID thresholds file.

<code lang="bash">!wget http://wiki.lrose.net/images/d/de/Pid_thresholds.sband.shv.txt</code>

2. Move the file to a better directory.

<code lang="bash">!mv ./Pid_thresholds.sband.shv.txt ${BASE_DIR}/params/nexrad_hca/</code>

</div>

<div class="alert alert-block alert-info"> <b>Task: set up the sounding information and link to the PID thresholds file.</b> 

1. Add the new path of the thresholds file to the parameter file: change <code lang="bash">pid_thresholds_file_path = "$(BASE_DIR)/params/nexrad_hca/Pid_thresholds.sband.shv.txt";</code>

2. Use the SPDB sounding profiles extracted from the model: change <code lang="bash">pid_override_temp_profile = TRUE;</code>

3. Add the path to the SPDB soundings: change <code lang="bash">sounding_spdb_url = "$(BASE_DIR)/data/nexrad_hca/sounding/spdb";</code>

4. Specify the 4-letter code for the radar (KCYS) sounding location: change <code lang="bash">sounding_location_name = "KCYS";</code>

</div>

---

Finally, we'll specify the output variables.

| Parameter | Description |
| --- | --- |
|output_fields|Set which output fields are written to the output files. The list of available output options are listed above the parameter.|

Of the fields calculated in RadxHca, we'll write out KDP, HCA, PID, and the attenuation-corrected fields (DBZ_ATTEN_CORRECTED, ZDR_ATTEN_CORRECTED). We'll copy a few of the input fields to the output files as well. 

<div class="alert alert-block alert-info"> <b>File Task: specify output fields.</b> 
    
In the RadxHca_params parameter file, write HCA, PID, KDP, DBZ_ATTEN_CORRECTED, and ZDR_ATTEN_CORRECTED in the <code lang="bash">output_fields</code> parameter. 

1. KDP is already included in the defaults, so no action needed for this variable.

2. For DBZ_ATTEN_CORRECTED and ZDR_ATTEN_CORRECTED, feel free to copy the entries for DBZ and ZDR from the defaults. Just append "\_ATTEN_CORRECTED" to the end of the <code lang="bash">id</code> and <code lang="bash">name</code> entries and "atten_corrected_" to the beginning of the <code lang="bash">long_name</code> and <code lang="bash">standard_name</code> entries.

3. For HCA and PID, create new entries either manually or by copying other variables. The <code lang="bash">id</code> and <code lang="bash">name</code> entries for HCA are both HCA, whereas the <code lang="bash">id</code> and <code lang="bash">name</code> entries for PID are PARTICLE_ID and PID, respectively. Feel free to use hydrometeor_type for the <code lang="bash">standard_name</code> entry for both variables. The <code lang="bash">long_name</code> entries can be "hydrometeor_classification" and "particle_id".

</div>


---

**Now we can run RadxHca!**

Here, *-params* provides the link to the main RadxHca parameter file, *-f* provides the link to the files we want to process, and *-outdir* indicates where RadxHca should write the final files.

In [ ]:
!RadxHca -params ${BASE_DIR}/params/nexrad_hca/RadxHca_params -f ${BASE_DIR}/data/nexrad_hca/cfradial/20250617/*.nc -outdir ${BASE_DIR}/data/nexrad_hca/hca


# 4. Plot HCA and PID for NEXRAD Cheyenne radar (KCYS)

To visualize the output in the notebook, we can use Py-ART. HawkEye is also great for visualizing data - a general parameter file can be found in the echo parameter files directory.

In [ ]:
# Read CfRadial file into radar object
inDir = base_dir+"/data/nexrad_hca/hca/20250617/"
file = "cfrad.20250617_082327.974_to_20250617_083013.086_KCYS_SUR.nc"
rate_kcys = pyart.io.read_cfradial(inDir+file)
rate_kcys.info('compact')


We can create a colormap for visualizing HCA and PID.

In [ ]:
#(17,15,10,11,12,9,6,4,5,7)

hcamap = np.array([[0.0, 0.0, 0.0, 1.0],
              [0.66015625, 0.66015625, 0.66015625, 1.0],
              [0.61960784313725492, 0.85490196078431369, 0.89803921568627454, 1.0],
              [0.090196078431372548, 0.74509803921568629, 0.81176470588235294, 1.0],
              [0.61176470588235299, 0.61960784313725492, 0.87058823529411766, 1.0],
              [1.0, 0.73333333333333328, 0.47058823529411764, 1.0],
              [0.83921568627450982, 0.15294117647058825, 0.15686274509803921, 1.0],
              [0.45490196078431372, 0.7686274509803922, 0.46274509803921571, 1.0],
              [0.17254901960784313, 0.62745098039215685, 0.17254901960784313, 1.0],
              [1.0, 0.59607843137254901, 0.58823529411764708, 1.0],],'f')
my_cmap3 = colors.ListedColormap(hcamap, name='nexrad_hca')


pidmap = np.array([[0.12156862745098039, 0.46666666666666667, 0.70588235294117652, 1.0],
              [0.68235294117647061, 0.7803921568627451, 0.90980392156862744, 1.0],
              [0.59607843137254901, 0.87450980392156863, 0.54117647058823526, 1.0],
              [0.45490196078431372, 0.7686274509803922, 0.46274509803921571, 1.0],
              [0.17254901960784313, 0.62745098039215685, 0.17254901960784313, 1.0],
              [0.83921568627450982, 0.15294117647058825, 0.15686274509803921, 1.0],
              [1.0, 0.59607843137254901, 0.58823529411764708, 1.0],
              [1.0, 0.49803921568627452, 0.054901960784313725, 1.0],
              [1.0, 0.73333333333333328, 0.47058823529411764, 1.0],
              [0.61960784313725492, 0.85490196078431369, 0.89803921568627454, 1.0],
              [0.090196078431372548, 0.74509803921568629, 0.81176470588235294, 1.0],
              [0.61176470588235299, 0.61960784313725492, 0.87058823529411766, 1.0],
              [0.32156862745098042, 0.32941176470588235, 0.63921568627450975, 1.0],
              [0.859375, 0.859375, 0.859375, 1.0],
              [0.66015625, 0.66015625, 0.66015625, 1.0],
              [0.41015625, 0.41015625, 0.41015625, 1.0],
              [0.0, 0.0, 0.0, 1.0],],'f')
my_cmap2 = colors.ListedColormap(pidmap, name='ncar_pid')


In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(rate_kcys)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('KDP', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# RATE_HYBRID (computed)

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('HCA', 0, vmin=0.5, vmax=10.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="HCA",
    cmap = my_cmap3)
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('PID', 0, vmin=0.5, vmax = 17.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="PID",
    cmap = my_cmap2, mask_outside=True)
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# plot all 17 PID categories
hca_cbar = displayRate.cbs[2]
hca_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10])
# (17,15,10,11,12,9,6,4,5,7)
hca_cbar.set_ticklabels(['clutter', 'biological', 'dry-snow', 'wet-snow', 'crystals', 'graupel', 'big-drops', 'lt-mod-rain', 'hvy-rain', 'rain/hail'])

pid_cbar = displayRate.cbs[3]
#pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13])
#pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice'])
pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice', 'slw', 'insects', '2nd-trip', 'clutter'])

figRate.tight_layout()

plt.show()


Next, we'll remake the same plot with the attenuation-corrected DBZ field.

In [ ]:
# Plot results of RadxRate, 

displayRate = pyart.graph.RadarDisplay(rate_kcys)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ_ATTEN_CORRECTED', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="Attenuation-Corrected DBZ")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('KDP', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# RATE_HYBRID (computed)

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('HCA', 0, vmin=0.5, vmax=10.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="HCA",
    cmap = my_cmap3)
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('PID', 0, vmin=0.5, vmax = 17.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="PID",
    cmap = my_cmap2, mask_outside=True)
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# plot all 17 PID categories
pid_cbar = displayRate.cbs[3]
#pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13])
#pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice'])
pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice', 'slw', 'insects', '2nd-trip', 'clutter'])

figRate.tight_layout()

plt.show()
